In [1]:
# # Install environment and agent
!pip install highway-env
# TODO: we use the bleeding edge version because the current stable version does not support the latest gym>=0.21 versions. Revert back to stable at the next SB3 release.
!pip install git+https://github.com/DLR-RM/stable-baselines3

# # Environment
import gymnasium as gym
import highway_env

# Agent
from stable_baselines3 import DQN


import sys
from tqdm.notebook import trange
!pip install tensorboardx gym pyvirtualdisplay
!apt-get install -y xvfb ffmpeg

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.3/954.3 kB 30.6 MB/s eta 0:00:00
  Cloning https://github.com/DLR-RM/stable-baselines3 to /tmp/pip-req-build-4z9wuvyg
  Running command git clone --filter=blob:none --quiet https://github.com/DLR-RM/stable-baselines3 /tmp/pip-req-build-4z9wuvyg
  Resolved https://github.com/DLR-RM/stable-baselines3 to commit 512eea923afad6f6da4bb53d72b6ea4c6d856e59
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 17.3 MB/s eta 0:00:00
  Created wheel for stable_baselines3: filename=stable_baselines3-2.4.0a9-py3-none-any.whl size=183041 sha256=aa0697ff7e06024c4a970011f524af3bbe805bb194f9475fdd20ea0fb95f360b
  Stored in directory: /tmp/pip-ephem-wheel-cache-u_m49zh6/wheels/3b/24/65/bc2794face336930a72bdbe36faf5aad6e2352b3d1dec310ca
Succ

In [2]:
from stable_baselines3 import DQN
import pprint
from matplotlib import pyplot as plt
import numpy as np

In [5]:
import base64
from pathlib import Path

from gymnasium.wrappers import RecordVideo
from IPython import display as ipythondisplay
from pyvirtualdisplay import Display



def record_videos(env, video_folder="videos"):
    wrapped = RecordVideo(
        env, video_folder=video_folder, episode_trigger=lambda e: True
    )

    # Capture intermediate frames
    env.unwrapped.set_record_video_wrapper(wrapped)

    return wrapped


def show_videos(path="videos"):
    html = []
    for mp4 in Path(path).glob("*.mp4"):
        video_b64 = base64.b64encode(mp4.read_bytes())
        html.append(
            """<video alt="{}" autoplay
                      loop controls style="height: 400px;">
                      <source src="data:video/mp4;base64,{}" type="video/mp4" />
                 </video>""".format(
                mp4, video_b64.decode("ascii")
            )
        )
    ipythondisplay.display(ipythondisplay.HTML(data="<br>".join(html)))


In [6]:
## record videos
def record_model_videos(env_id, model_name, vehicle_count=10, episodes=3, video_folder='videos'):
    # Environment configuration
    config = {
        "observation": {
            "type": "Kinematics",
            "features": ["presence", "x", "y", "vx", "vy"],
            "absolute": True,
            "normalize": True,
            "vehicles_count": vehicle_count,
            "see_behind": True,
        },
        "action": {
            "type": "DiscreteMetaAction",
            "target_speeds": np.linspace(0, 32, 9),
        },
        "duration": 40,
        "vehicles_density": 2,
        "show_trajectories": True,
        "render_agent": True,
    }
    ##load model
    model = DQN.load(f"models/{model_name}")

    # Create environment
    env = gym.make(env_id, render_mode='rgb_array')
    env.configure(config)

    # Use RecordVideo wrapper to record videos in the specified folder
    video_path = f"{video_folder}/{model_name}"
    env = RecordVideo(env, video_folder=video_path, episode_trigger=lambda ep: True)

    # Loop through the episodes
    for episode in trange(episodes, desc=f'Testing {model_name}'):
        (obs, info), done, truncated = env.reset(), False, False
        while not (done or truncated):
            # Replace rf_query with your model prediction function to get action
            action, _ = model.predict(obs, deterministic=True)
            obs, reward, done, truncated, info = env.step(int(action))

    # Close the environment after recording
    env.close()

    print(f"Videos saved for {model_name} in {video_path}")

In [7]:
from stable_baselines3 import DQN
import pprint
from matplotlib import pyplot as plt
import numpy as np

llm stuff


PHI3


In [8]:
!pip install -q transformers huggingface_hub[cli] bitsandbytes accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 8.7 MB/s eta 0:00:00


In [9]:
import torch
from transformers import BitsAndBytesConfig

config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)

In [10]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

torch.random.manual_seed(0)

llm_model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
    quantization_config = config
)

tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/11.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.2k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/3.44k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

In [11]:
def llm_action(prompt1, assist1, prompt2, last_act='FASTER'):

    messages = [{"role": "user", "content": prompt1},
           {"role": "assistant", "content": assist1},
           {"role": "user", "content": prompt2}]

    model_inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")

    output = llm_model.generate(model_inputs, max_new_tokens=2000, do_sample=True)

    decoded_output = tokenizer.batch_decode(output[:,model_inputs.size(1):], skip_special_tokens=True)

    try:
        action = decoded_output[0].strip().split('Final decision: ')[1].strip().split('\'')[0]
    except:
        action = last_act

    return action

In [12]:
action_dict = {
    0: 'LANE_LEFT',
    1: 'IDLE',
    2: 'LANE_RIGHT',
    3: 'FASTER',
    4: 'SLOWER'
}

In [13]:
class MyHighwayEnv_llm(gym.Env):
    def __init__(self, vehicleCount=10):
        super(MyHighwayEnv_llm, self).__init__()
        # base setting
        self.vehicleCount = vehicleCount
        self.prev_action  = 'FASTER'

        # environment setting
        self.config = {
            "observation": {
                "type": "Kinematics",
                "features": ["presence", "x", "y", "vx", "vy"],
                "absolute": True,
                "normalize": False,
                "vehicles_count": vehicleCount,
                "see_behind": True,
            },
            "action": {
                "type": "DiscreteMetaAction",
                "target_speeds": np.linspace(0, 32, 9),
            },
            "duration": 40,
            "vehicles_density": 2,
            "show_trajectories": True,
            "render_agent": True,
        }
        self.env = gym.make("highway-v0")
        self.env.configure(self.config)
        self.action_space = self.env.action_space
        self.observation_space = gym.spaces.Box(
            low=-np.inf,high=np.inf,shape=(10,5),dtype=np.float32
        )

    def find_smallest_positive(self, arr):
        smallest_positive = float('inf')
        index = -1

        for i, value in enumerate(arr):
            if 0 < value < smallest_positive:
                smallest_positive = value
                index = i

        return smallest_positive, index

    def prompt_design(self, obs_):

        prompt1 = 'You are a smart driving assistant. You, the \'ego\' car, are now driving on a highway. You need to recommend ONLY ONE best action among the following set of actions based on the current scenario: \n \
        \t1. IDLE -- maintain the current speed in the current lane \n \
        \t2. FASTER -- accelerate the ego vehicle \n \
        \t3. SLOWER -- decelerate the ego vehicle \n \
        \t4. LANE_LEFT -- change to the adjacent left lane \n \
        \t5. LANE_RIGHT -- change to the adjacent right lane\n'

        assist1 = 'Understood. Please provide the current scenario or conditions, such as traffic density, speed of surrounding vehicles, your current speed, and any other relevant information, so I can recommend the best action.'

        prompt2 = 'Here is the current scenario:\n \
        There are four lanes on the highway: Lane-1 (left most), Lane-2, Lane-3, Lane-4 (right most). \n\n'

        x, y, vx, vy = obs_[:,1], obs_[:,2], obs_[:,3], obs_[:,4]

        ego_x, ego_y   = x[0], y[0]
        ego_vx, ego_vy = vx[0], vy[0]

        veh_x, veh_y   = x[1:] - ego_x, y[1:] - ego_y
        veh_vx, veh_vy = vx[1:], vy[1:]

        lanes          = y//4+1
        ego_lane       = lanes[0]
        veh_lanes      = lanes[1:]

        if ego_lane == 1:
            ego_left_lane  = 'Left lane: Not available\n'
            ego_right_lane = 'Right lane: Lane-' + str(ego_lane+1) + '\n'
        elif ego_lane == 4:
            ego_left_lane  = 'Left lane: Lane-' + str(ego_lane-1) + '\n'
            ego_right_lane = 'Right lane: Not available\n'
        else:
            ego_left_lane  = 'Left lane: Lane-' + str(ego_lane-1) + '\n'
            ego_right_lane = 'Right lane: Lane-' + str(ego_lane+1) + '\n'

        prompt2 += 'Ego vehicle:\n \
        \tCurrent lane: Lane-' + str(ego_lane) + '\n' + '\t' + ego_left_lane + '\t' + ego_right_lane + '\tCurrent speed: ' + str(ego_vx) + ' m/s \n\n'

        lane_info = 'Lane info:\n'
        for i in range(4):
            inds     = np.where(veh_lanes == i+1)[0]
            num_v    = len(inds)
            if num_v > 0:
                val, ind = self.find_smallest_positive(veh_x[inds])
                true_ind = inds[ind]
                lane_info += '\tLane-' + str(i+1) + ': There are ' + str(num_v) + ' vehicle(s) in this lane ahead of ego vehicle, closest being ' + str(veh_x[true_ind]) + ' m ahead traveling at ' + str(veh_vx[true_ind]) + ' m\/s. \n'
            else:
                lane_info += '\tLane-' + str(i+1) + ' No other vehicle ahead of ego vehicle.\n'

        prompt2 += lane_info

        att_info = '\nAttention points:\n \
        \t1. SLOWER has least priority and should be used only when no other action is safe.\n \
        \t2. DO NOT change lanes frequently.\n \
        \t3. Safety is priority, but do not forget efficiency.\n \
        \t4. Your suggested action has to be one from one of the above five listed actions - IDLE, SLOWER, FASTER, LANE_LEFT, LANE_RIGHT. \n \
        Your last action was ' + self.prev_action + '. Please recommend action for the current scenario ONLY in the format \'Final decision: <final decision>\'.\n'

        prompt2 += att_info


        return prompt1, assist1, prompt2

    def prompt_design_blog(self, obs_):

      # Part 1: Initial prompt introducing the scenario
      prompt1 = 'You are Phi3, a large language model. You are now acting as a mature driving assistant, who can give accurate and correct advice for human drivers in complex urban driving scenarios. The information in the current scenario:\n\
                You, the \'ego\' car, are now driving on a highway. You have already driven for 0 seconds.\n\
                The decision made by the agent LAST time step was \'FASTER\' (accelerate the vehicle).'

      # Part 2: Driving rules that must be followed
      rules = "There are several rules you need to follow when you drive on a highway:\n\
              1. Try to keep a safe distance from the car in front of you.\n\
              2. DON’T change lanes frequently. If you want to change lanes, double-check the safety of vehicles in the target lane."

      prompt1 += rules

      # Part 3: Attention points for decision making
      att_points = "Here are your attention points:\n\
                    1. You must output a decision when you finish this task. Your final output decision must be unique and not ambiguous. For example, you cannot say \"I can either keep lane or accelerate at the current time\".\n\
                    2. You need to always remember your current lane ID, your available actions, and available lanes before you make any decision.\n\
                    3. Once you have a decision, you should check the safety with all the vehicles affected by your decision.\n\
                    4. If you verify a decision is unsafe, you should start a new one and verify its safety again from scratch."

      prompt1 += att_points

      # Part 4: Request for additional scenario details
      assist1 = 'Understood. Please provide the current scenario or conditions, such as traffic density, speed of surrounding vehicles, your current speed, and any other relevant information, so I can recommend the best action.'

      # Part 5: Describing the current highway scenario
      prompt2 = 'Here is the current scenario:\n\
      There are four lanes on the highway: Lane-1 (leftmost), Lane-2, Lane-3, Lane-4 (rightmost).\n\n'

      # Extract information from the observations
      x, y, vx, vy = obs_[:, 1], obs_[:, 2], obs_[:, 3], obs_[:, 4]

      # Ego vehicle details
      ego_x, ego_y = x[0], y[0]
      ego_vx, ego_vy = vx[0], vy[0]

      # Other vehicles' relative positions
      veh_x, veh_y = x[1:] - ego_x, y[1:] - ego_y
      veh_vx, veh_vy = vx[1:], vy[1:]

      # Determine lane information for ego and other vehicles
      lanes = y // 4 + 1
      ego_lane = lanes[0]
      veh_lanes = lanes[1:]

      # Left and right lane availability based on the ego vehicle's current lane
      if ego_lane == 1:
          ego_left_lane = 'Left lane: Not available\n'
          ego_right_lane = 'Right lane: Lane-' + str(ego_lane + 1) + '\n'
      elif ego_lane == 4:
          ego_left_lane = 'Left lane: Lane-' + str(ego_lane - 1) + '\n'
          ego_right_lane = 'Right lane: Not available\n'
      else:
          ego_left_lane = 'Left lane: Lane-' + str(ego_lane - 1) + '\n'
          ego_right_lane = 'Right lane: Lane-' + str(ego_lane + 1) + '\n'

      # Append ego vehicle information to prompt2
      prompt2 += 'Ego vehicle:\n\
      \tCurrent lane: Lane-' + str(ego_lane) + '\n' + '\t' + ego_left_lane + '\t' + ego_right_lane + '\tCurrent speed: ' + str(ego_vx) + ' m/s\n\n'

      # Lane information including vehicles ahead in each lane
      lane_info = 'Lane info:\n'
      for i in range(4):
          inds = np.where(veh_lanes == i + 1)[0]
          num_v = len(inds)
          if num_v > 0:
              # Find the closest vehicle in the current lane
              val, ind = self.find_smallest_positive(veh_x[inds])
              true_ind = inds[ind]
              lane_info += '\tLane-' + str(i + 1) + ': There are ' + str(num_v) + ' vehicle(s) in this lane ahead of ego vehicle, closest being ' + str(veh_x[true_ind]) + ' m ahead traveling at ' + str(veh_vx[true_ind]) + ' m/s.\n'
          else:
              lane_info += '\tLane-' + str(i + 1) + ': No other vehicle ahead of ego vehicle.\n'

      # Append lane information to prompt2
      prompt2 += lane_info

      # Part 6: Adding additional attention points and the final decision instruction
      att_info = '\nAttention points:\n\
      \t1. SLOWER has the least priority and should be used only when no other action is safe.\n\
      \t2. DO NOT change lanes frequently.\n\
      \t3. Safety is a priority, but do not forget efficiency.\n\
      \t4. Your suggested action has to be one from the five listed actions - IDLE, SLOWER, FASTER, LANE_LEFT, LANE_RIGHT.\n\
      Your last action was ' + self.prev_action + '. Please recommend action for the current scenario ONLY in the format \'Final decision: <final decision>\'.\n'

      # Append the attention information to prompt2
      prompt2 += att_info

      # Return the three prompts
      return prompt1, assist1, prompt2



    def step(self, action):

        # Step the wrapped environment and capture all returned values
        obs, dqn_reward, done, truncated, info = self.env.step(action)

        self.prev_action = action_dict[action]

        Reward = 1 / (1 + np.exp(-dqn_reward))

        return obs, Reward, done, truncated, info

    def reset(self, **kwargs):
        obs = self.env.reset(**kwargs)
        return obs  # Make sure to return the observation



<>:91: DeprecationWarning: invalid escape sequence '\/'


GENERATE DATASET

In [14]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import joblib
from sklearn.metrics import accuracy_score
import pandas as pd
import os

In [17]:
import os
import numpy as np
import pandas as pd
from tqdm import trange
import random

def generate_dataset_model(env, model, file_name, episodes=500):
    observations = []
    actions = []

    for episode in trange(episodes, desc="Dataset Generation"):
        (obs, info), done, truncated = env.reset(), False, False
        while not (done or truncated):
            # Generate prompt for LLM
            prompt1, assist1, prompt2 = env.prompt_design_blog(obs)
            llm_act = llm_action(prompt1, assist1, prompt2, env.prev_action).strip().split('.')[0]

            # Convert LLM action to a numerical label
            if 'LANE_LEFT' in llm_act:
                action_label = 0
            elif 'IDLE' in llm_act:
                action_label = 1
            elif 'LANE_RIGHT' in llm_act:
                action_label = 2
            elif 'FASTER' in llm_act:
                action_label = 3
            elif 'SLOWER' in llm_act:
                action_label = 4

            # Store observation and corresponding LLM action
            observations.append(obs.flatten())
            actions.append(action_label)

            # Predict action using model and step the environment
            action, _ = model.predict(obs, deterministic=True)
            obs, reward, done, truncated, info = env.step(int(action))

    # Convert to numpy arrays
    observations = np.array(observations)
    actions = np.array(actions)

    # Save the dataset as a CSV file
    data = pd.DataFrame(observations)
    data['action'] = actions

    # Create a directory to save the dataset if it doesn't exist
    dataset_dir = 'datasets'
    if not os.path.exists(dataset_dir):
        os.makedirs(dataset_dir)

    # Save the dataset in the Colab workspace
    dataset_path = os.path.join(dataset_dir, file_name)
    data.to_csv(dataset_path, index=False)

    print(f"Dataset saved to {dataset_path}")

    return observations, actions

def generate_dataset_epsilon(env, model, file_name, episodes=50, epsilon_start=1.0, epsilon_min=0.1, epsilon_decay=0.99):
    observations = []
    actions = []
    epsilon = epsilon_start

    for episode in trange(episodes, desc="Dataset Generation"):
        (obs, info), done, truncated = env.reset(), False, False
        while not (done or truncated):
            # Epsilon-greedy action selection
            if random.uniform(0, 1) < epsilon:
                # Take a random action (exploration)
                action = env.action_space.sample()
            else:
                # Predict action using the model (exploitation)
                action, _ = model.predict(obs, deterministic=True)

            # Generate prompt for LLM
            prompt1, assist1, prompt2 = env.prompt_design_blog(obs)
            llm_act = llm_action(prompt1, assist1, prompt2, env.prev_action).strip().split('.')[0]

            # Convert LLM action to a numerical label
            if 'LANE_LEFT' in llm_act:
                action_label = 0
            elif 'IDLE' in llm_act:
                action_label = 1
            elif 'LANE_RIGHT' in llm_act:
                action_label = 2
            elif 'FASTER' in llm_act:
                action_label = 3
            elif 'SLOWER' in llm_act:
                action_label = 4

            # Store observation and corresponding LLM action
            observations.append(obs.flatten())
            actions.append(action_label)

            # Step the environment with the chosen action
            obs, reward, done, truncated, info = env.step(int(action))

            # Decay epsilon after each episode to reduce exploration over time
            epsilon = max(epsilon_min, epsilon * epsilon_decay)

    # Convert to numpy arrays
    observations = np.array(observations)
    actions = np.array(actions)

    # Save the dataset as a CSV file
    data = pd.DataFrame(observations)
    data['action'] = actions

    # Create a directory to save the dataset if it doesn't exist
    if not os.path.exists('datasets'):
        os.makedirs('datasets')

    # Save the dataset locally
    dataset_path = os.path.join('datasets', file_name)
    data.to_csv(dataset_path, index=False)

    print(f"Dataset saved to {dataset_path}")

    return observations, actions


In [ ]:
# Create and configure the environment
env = MyHighwayEnv_llm()

modelDqn = DQN.load("dqn_model_default_fast")

# Generate dataset from environment and LLM
observations, actions = generate_dataset_epsilon(env, modelDqn, "highway_groq.csv")

# Train Random Forest Model
# save_path = 'models/random_forest_model_new.pkl'
# rf_model = train_xgb_model(observations, actions, save_path)


/usr/local/lib/python3.10/dist-packages/gymnasium/core.py:311: UserWarning: WARN: env.configure to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.configure` for environment variables or `env.get_wrapper_attr('configure')` that will search the reminding wrappers.
  logger.warn(
/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/save_util.py:167: UserWarning: Could not deserialize object lr_schedule. Consider using `custom_objects` argument to replace this object.
Exception: Can't get attribute '_function_setstate' on <module 'cloudpickle.cloudpickle' from '/usr/local/lib/python3.10/dist-packages/cloudpickle/cloudpickle.py'>
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/save_util.py:167: UserWarning: Could not deserialize object exploration_schedule. Consider using `custom_objects` argument to replace this object.
Exception: Can't get attribute '_function_setstat